# Tag에 대한 인기글 상세정보 가져오기
### Tag, img, tags, view, love, tags_cnt

In [72]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from selenium import webdriver
from multiprocessing import Pool
from pymongo import MongoClient

import io
import json
import re
import pandas as pd
import random
import pymongo
import json
import io

class InstarSelenium:
    def __init__(self): 
        self.driver = webdriver.Chrome('chromedriver_win32/chromedriver') # 크롬 드라이버 실행.
        self.driver.implicitly_wait(3) # 대기시간.
        self.driver.get('https://www.instagram.com/explore/') # 크롬 실행시 맨 처음뜨는 Page.
        
    
    def login(self, username, passward):
        self.driver.find_element_by_name('username').send_keys(username) # Id 값.
        self.driver.find_element_by_name('password').send_keys(passward) # Page 값.
        
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/button').click() # 로그인 버튼 클릭.
        delay = 3
        try:
            login = WebDriverWait(self.driver, delay).until(EC.presence_of_element_located((By.XPATH, "//*[@id='react-root']/section/nav/div[2]/div/div/div[2]/input"))) # 로그인 창 생길 때 까지 기다림.
            if login != None:
                print("로그인 성공!") 
        except:
            print("로그인 실패...")
            
    def get_tags_list(self, filename): # txt에 있는 tag값 읽기.
        with io.open(filename, 'r', encoding='utf-8-sig') as f: # \값이 안에있으면 json 안읽힘...ㅠㅠ흐규흐규
            for line in f: 
                tweet = json.loads(line)
        return pd.DataFrame(tweet) # DataFrame으로 변환 후 리턴.
    
    def fast_data_from_tags(self, tags):
        # DB연결.
        conn = MongoClient('192.168.0.5', 27017) # MongoClient를 이용해 DB연결
        instar = conn.instar # Collections 연결
        #tag = instar.tag #  Collection 연결
        tagdetail = instar.tagdetail
        #client = MongoClient('mongodb://127.0.0.1:27017') # Url형식으로 데이터베이스 주소 설정.

        #result = {'love':[], 'view':[], 'tags':[], 'tags_cnt':[], 'img':[]}
          
        # 코드 실행
        amount = len(tags)
        count = 0
        #result = {'tag':[], 'love':[], 'view':[], 'tags':[], 'tags_cnt':[], 'img':[]}
        
        # tag 입력
        delay = 5
        for idx, tag in enumerate(tags):
#             tagdb = tagdetail.distinct(key="tag", filter={"tag":tag}) # Mongo DB에 tag들에 중복 값 제거.
#             tagdb = "".join(tagdb) # List를 String으로 변환.
#             if (tag == tagdb): # 같은 Tag가 있으면 넘김. (이미 DB에 있는 값이기 때문에..)
#                 continue
            
            self.driver.get("https://www.instagram.com/explore/tags/" + tag + "/") # tag값에 따라 사이트 이동.
            try:
                hot_contents = WebDriverWait(self.driver, delay).until(EC.presence_of_element_located((By.XPATH, "//*[@id='react-root']/section/main/article/div[1]/div/div"))) # div.react-root Tag 생길 때 까지 대기.
            except TimeoutException:
                continue # 오류시 다음으로 넘김.
            
#             for content in hot_contents.find_elements_by_class_name('_9AhH0'):
#                 content.click()
#                 self.driver.back()
            
            for content in hot_contents.find_elements_by_class_name('_9AhH0'):
            # love : 태그이름
            # view : 태그게시물수
            # tags : 전체태그
            # tags_cnt : 태그 갯수
            # img : 이미지 Link
                love = ""
                view = ""
                tagsdb = ""
                tags_cnt = ""
                img = []
            
                content.click()
                try:
                    article = WebDriverWait(self.driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div:nth-of-type(3) > div > div.zZYga > div > article'))) # article Tag 생길 때 까지 대기.
                    
                    html = article.get_attribute('innerHTML') # Html 읽기.
                    soup = BeautifulSoup(html, 'html.parser') # soup으로 변환.

                    love = soup.select_one('div.eo2As > section.EDfFK.ygqzn > div > div > button') # love tag 찾기.
                    #view = soup.select_one('div.eo2As > section.EDfFK.ygqzn > div > span') # 
                    view = soup.select_one('div.eo2As') # view tag 찾기.
                    imgs = soup.findAll('img', {'class':['_8jZFn','FFVAD']}) # img 태그 찾기.
                    if imgs != None: # 이미지가 있으면.
                        img_list = [] # 이미지 리스트 변수.
                        for img in imgs: # 이미지가 있는 만큼 돌리기.
                            if img != None: # 이미지가 있을 때 마다.
                                img_list.append(img.get('src')) # img_list에 추가.
                        #result['img'].append(img_list)
                        img = img_list # DB에 넣을 변수에 저장.
                            
                    if love: # love가 있으면.
                        if love.find('span'):
                            #result['love'].append(love.find('span').text)
                            love = love.find('span').text # 안에있는 좋아요 갯수 가져오기.
                        else: # 위에 없을 경우.
                            #result['love'].append(re.findall("[\d,]+", love.text))
                            love = re.findall("[\d,]+", love.text) # 다른 태그에 있는 좋아요.
                        
                        #result['view'].append('0')
                        view = '0'
                    else: # love가 없으면 view 찾기.
                        #result['love'].append('0')
                        love = '0'
                        if view.find('section.ltpMr Slqrh'): # 없으면 pass
                            pass
                        elif view.find('section.EDfFK.ygqzn > div > span > span'): # 있으면 동영상 view수 가져오기.
                            #result['view'].append(view.text)
                            view = view.text
                        else:
                            #result['view'].append(view.find('span').text)
                            view = view.find('span').text # 위에 없으면 span에 있는 동영샹 view수 가져오기.
                            
                    content1 = soup.select_one('div.eo2As > div.KlCQn.EtaWk > ul.k59kT > li > div > div > div > span') # 태그들 값 가져오기 1
        
                                                
                    if content1:
                        text = re.sub('<.+?>', ' ', str(content1), 0).strip() # 태그값 가져오기. 공백제거
                        tags1 = re.compile('#[a-zA-Z가-힣]+').findall(text) # txet에서 # 값만 가져오기.
                                            
                    
                    #result['tags'].append(tags)
                    #result['tags_cnt'].append(len(tags))
#                     tagsdb = tags1 + tags2 # 태그값 더해서
                    tags_cnt = len(tagsdb) # DB 변수에 넣기.
                    
#                     global tagsdb
#                     tagsdb = tags1 + tags2
#                     tags_cnt = len(tagsdb)
                        
                except TimeoutException:
                    print("Loading took too much time!")               
                    count += 1
                    
                self.driver.back()
                
                if count >= 3: # Lock 3번이상 걸리면.
                    self.driver.quit() # 종료 하기.
                    
                    self.driver = webdriver.Chrome('C:/Users/kitcoop/Downloads/chromedriver_win32/chromedriver') # 드라이버 다시 실행.
                    self.driver.implicitly_wait(3)
                    
                    count = 0
                    break
                    
                #result['tag'].append(tag)
#                 tagdetail.insert({"tag": tag, "love": love, "view": view, "tags": tagsdb, "tags_cnt": tags_cnt, "img": img}) # 태그 값 DB에 삽입.
                
                print("좋아요: ", love)
                print("동영상 뷰: ", view)
                print("태그: ", " ".join(tags1))
                print("태그수: ", len(tags1))
                print("이미지 URL: ", img[0])
    
            if idx+1 != amount:
                print(round((idx+1) / amount * 100, 2), "% 진행중") # 진행 %.
            else:
                print('완료!')
                
        return ""

# Instar 객체 생성 및 Instargram 로그인

In [73]:
instar = InstarSelenium()

# 전체 Tag 값 가져오기
### txt에 있는 tag가져와 변수에 저장

In [75]:
df_tags = instar.get_tags_list('hashtag222.txt') # txt 값 가져오기.
df_tags['tagname'] = df_tags['tagname'].str[1:]  # # 제거(주소값에는 # 불필요)
df_tags['tagcount'] = df_tags['tagcount'].str.replace(",","").astype(int)
df_tags = df_tags[df_tags['tagcount'] > 4]
df_tags = df_tags.sort_values('tagcount', ascending=False)[0:3]
df_tags.shape # tag shape
df_tags

,tagcount,tagname
91423,151955143,일상
32263,91253613,데일리
46595,87632298,맞팔


# Tag Detail(상세정보 가져오기)

In [76]:
import time
start_time = time.time() 

# pool = Pool(processes=4)
# imsi = pool.map()
instar.fast_data_from_tags(df_tags['tagname'])
print("--- %s seconds ---" %(time.time() - start_time))

좋아요:  975
동영상 뷰:  0
태그:  
태그수:  0
이미지 URL:  https://scontent-icn1-1.cdninstagram.com/vp/57b337933e92e50681e2a84ffb1e318b/5CC175B0/t51.2885-15/e35/47692278_704002043317591_881738031988975617_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
좋아요:  109
동영상 뷰:  0
태그:  #일상 #kids
태그수:  2
이미지 URL:  https://scontent-icn1-1.cdninstagram.com/vp/2147abfd0aadb40f382120a9d4381d8f/5CD36D48/t51.2885-15/e35/46844502_292008704838574_8111545919139926399_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
좋아요:  2,021
동영상 뷰:  0
태그:  #제주도 #모실클럽하우스 #수영장 #방학그램 #육아그램 #일상 #소통
태그수:  7
이미지 URL:  https://scontent-icn1-1.cdninstagram.com/vp/77fb8da00579c5b3d1290397b18eba4b/5CCC49FA/t51.2885-15/e35/47691006_360604268073336_1564466293583202017_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com
좋아요:  210
동영상 뷰:  0
태그:  #세이펜 #공구 #예정 #언제나최저가
태그수:  4
이미지 URL:  https://scontent-icn1-1.cdninstagram.com/vp/d25463f49f77ff76a9daf97af03c022a/5CD8E069/t51.2885-15/e35/47251278_2161458047427197_966710729672881624_n.jpg?_nc_ht=scontent-icn1-1.c

# Mongo DB에서 값 읽기

In [21]:
# DB연결.
conn = MongoClient('127.0.0.1', 27017) # MongoClient를 이용해 DB연결
instar = conn.instar # Collections 연결
#tag = instar.tag #  Collection 연결
tagdetail = instar.tagdetail

for i in tagdetail.find({},{ "_id": 0, "tag": 1}): # 제외하고싶은 값 0, 필요한 값은 1
    print(i)

tag = tagdetail.find()
#.distinct(key="tag", filter={"tag":"일상"})
for i in tagdetail.find():
    print(i)